# PCE (Personal Consumption Expenditures) Explorer

Deep-dive into PCE components, price indexes, and statistical analysis.

In [ ]:
from macro_econ.series import build_pce_tree
from macro_econ.clients import FredClient
from macro_econ.transforms import (
    mom_change, mom_annualized, yoy_change, n_month_annualized,
    moving_average, adf_test, kpss_test, stl_decompose, compute_acf_pacf
)
from macro_econ.viz.charts import line_chart, stacked_bar_contributions, stl_plot, acf_pacf_plot, recession_shading
from macro_econ.viz.widgets import build_tree_widget, transform_selector

fred = FredClient()
pce = build_pce_tree()
print(pce.print_tree())

## PCE Hierarchy Browser

In [ ]:
import ipywidgets as widgets
from IPython.display import display

output = widgets.Output()

def on_node_select(node):
    with output:
        output.clear_output()
        src = node.get_source('fred')
        if src:
            df = fred.fetch_series(src.series_id, start_date='2000-01-01')
            fig = line_chart({node.name: df}, f'{node.name} ({src.series_id})')
            fig = recession_shading(fig)
            fig.show()
        else:
            print(f'No FRED source for {node.name}')

tree_widget = build_tree_widget(pce, on_select=on_node_select)
display(tree_widget, output)

## Goods vs Services Split

In [ ]:
# Fetch major components
pce_total = fred.fetch_series('PCE', start_date='2000-01-01')
goods = fred.fetch_series('DGDSRC1M027NBEA', start_date='2000-01-01')
services = fred.fetch_series('PCES', start_date='2000-01-01')

fig = line_chart(
    {'Total PCE': pce_total, 'Goods': goods, 'Services': services},
    'PCE: Total, Goods, and Services (Billions $)',
    yaxis_title='Billions of Dollars'
)
fig = recession_shading(fig)
fig.show()

## PCE Price Index: Headline vs Core

In [ ]:
pce_pi = fred.fetch_series('PCEPI', start_date='2000-01-01')
pce_core = fred.fetch_series('PCEPILFE', start_date='2000-01-01')

# YoY
fig = line_chart(
    {'PCE YoY': yoy_change(pce_pi, 12), 'Core PCE YoY': yoy_change(pce_core, 12)},
    'PCE Inflation: Headline vs Core (YoY %)'
)
fig = recession_shading(fig)
fig.show()

# 3m and 6m annualized
fig = line_chart(
    {
        'Core PCE 3m Ann.': n_month_annualized(pce_core, 3),
        'Core PCE 6m Ann.': n_month_annualized(pce_core, 6),
        'Core PCE YoY': yoy_change(pce_core, 12),
    },
    'Core PCE Inflation: Multi-Horizon View'
)
fig.show()

## Statistical Analysis

In [ ]:
# Stationarity tests on Core PCE YoY
core_yoy = yoy_change(pce_core, 12).dropna()

adf = adf_test(core_yoy)
kpss_res = kpss_test(core_yoy)
print(adf.summary)
print(kpss_res.summary)

# ACF / PACF
acf_vals, pacf_vals = compute_acf_pacf(core_yoy, nlags=24)
fig = acf_pacf_plot(acf_vals, pacf_vals, 'Core PCE YoY: ACF and PACF')
fig.show()

# STL Decomposition
stl_result = stl_decompose(pce_core['value'], period=12)
fig = stl_plot(stl_result, 'Core PCE Price Index: STL Decomposition')
fig.show()